# Module 1, Section 3: Multi-Agent Architecture


In this section, you'll learn to build a multi-agent customer support system using:
- **Specialized sub-agents** focused on distinct domains (database vs. documents)
- **Supervisor agent** that smartly routes queries to the right expert
- **Tool wrapping**, enabling the supervisor to delegate tasks to sub-agents as tools
- **Testing with LangSmith traces** to see multi-agent coordination in action

By the end, you'll have a working system with:
- **Database Agent** for order, product, and customer queries
- **Documents Agent** for searching product documents and policies
- **Supervisor** for orchestration and delegation

<div align="center">
    <img src="../../static/supervisor_agent.png">
</div>


## Setup

Load environment variables:

In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 1. Import Tools

We'll use the tools we created in previous sections, plus new Documents tools for document search:

<div align="center">
    <img src="../../static/db_rag_tools.png" alt="Schema Diagram">
</div>

In [ ]:
# Import database tools (from Section 1)
from tools import get_order_details, get_product_price

# Import Documents tools (new in Section 3)
from tools import search_product_docs, search_policy_docs

### What is RAG (Retrieval-Augmented Generation)?

Both our Database and Documents agents use retrieval patterns - they retrieve information to augment the LLM's context. The Documents Agent specifically uses **RAG with semantic search** over unstructured documents:

1. **Documents are split into chunks** (~1000 characters)
2. **Chunks become vectors (embeddings)** that capture meaning
3. **User query becomes a vector** too
4. **Similar vectors = similar meaning** → retrieve relevant chunks
5. **LLM uses retrieved chunks** to answer the question

**Our Setup:**
- **25 product docs + 5 policy docs** → **337 chunks**
- **HuggingFace embeddings** (local, no API key needed)
- **Metadata filtering** separates products from policies
- **Pre-built vectorstore** loaded from `data/vector_stores/`

The Documents tools (`search_product_docs` and `search_policy_docs`) wrap this complexity into simple function calls!

In [3]:
result = search_policy_docs.invoke("How long is the warranty on electronics?")
print(result)

[warranty_guide]
# Warranty Guide

All products purchased from TechHub include manufacturer warranty coverage. This guide explains what's covered, what's not, and how to file a warranty claim.

## Standard Warranty Coverage

**All Products Include:**
- 1-year limited manufacturer warranty
- Coverage begins on the delivery date
- Covers manufacturing defects in materials and workmanship
- Hardware component failures under normal use conditions

**What This Means:**
Your product is guaranteed to be free from defects in materials and workmanship for one year from the date of delivery. If a covered issue occurs during this period, the manufacturer will repair or replace the product at no charge.

## What's Covered

The manufacturer warranty covers:

---

[warranty_guide]
## How TechHub Can Help

While manufacturers handle warranty service directly, we're here to assist:

- **Proof of Purchase:** We can provide your receipt or invoice for warranty claims
- **Facilitation:** We can help you 

## 2. Build Database Agent

Our first specialist: an agent focused on querying structured data (orders, products, customers).

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver

# Initialize model
llm = init_chat_model("anthropic:claude-haiku-4-5")

# Create Database Agent
db_agent = create_agent(
    model=llm,
    tools=[get_order_details, get_product_price],
    system_prompt="""You are a database specialist for TechHub customer support.
    
Your role is to query:
- Order status and details
- Product prices and availability
- Customer order history

Always provide specific, accurate information from the database.
If you cannot find information, say so clearly.""",
    checkpointer=MemorySaver(),
)

Let's test the Database Agent:

In [5]:
import uuid

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}


result = db_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What items were in order ORD-2024-0063?"}
        ]
    },
    config=config,
)

print(result["messages"][-1].content)

Order ORD-2024-0063 contained the following item:

- **Dell UltraSharp 27" 4K Monitor** (Quantity: 1) - $543.74

The order has been **Delivered** on July 14, 2024, with tracking number 1Z999AA192518345.


## 3. Build Documents Agent

Our second specialist: an agent focused on searching product documentation and policies.

In [6]:
# Create Documents Agent
docs_agent = create_agent(
    model=llm,
    tools=[search_product_docs, search_policy_docs],
    system_prompt="""You are a company policy and product information specialist for TechHub customer support.

Your role is to answer questions about product specifications, features, compatibility,
policies (returns, warranties, shipping), and setup instructions.

Always search the documentation to provide accurate, detailed information.
If you cannot find information, say so clearly.""",
    checkpointer=MemorySaver(),
)

Let's test the Documents Agent:

In [7]:
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = docs_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What's your return policy for opened electronics?",
            }
        ]
    },
    config=config,
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's your return policy for opened electronics?
================================== Ai Message ==================================

[{'id': 'toolu_01DFg8hEcRrWC8t8EY2b7qXE', 'input': {'query': 'return policy opened electronics'}, 'name': 'search_policy_docs', 'type': 'tool_use'}]
Tool Calls:
  search_policy_docs (toolu_01DFg8hEcRrWC8t8EY2b7qXE)
 Call ID: toolu_01DFg8hEcRrWC8t8EY2b7qXE
  Args:
    query: return policy opened electronics
================================= Tool Message =================================
Name: search_policy_docs

[return_policy]
# Return Policy

At TechHub, we want you to be completely satisfied with your purchase. If you're not happy with your order, we accept returns within our specified return windows.

## Return Windows

**Unopened Electronics**
- 30 days from delivery date
- All original packaging and seals must be intact
- Full refund to original payment method

**Opened 

## 4. Build Supervisor Agent

Now we'll create a supervisor agent that coordinates our specialists.

**Key insight:** Sub-agents become *tools* for the supervisor!

In [8]:
from langchain_core.tools import tool


# Wrap Database Agent as a tool
@tool(
    "database_specialist",
    description="Query TechHub database for order status, product prices, and customer order history",
)
def call_database_specialist(query: str) -> str:
    """Call the database specialist subagent.

    Args:
        query: The question to ask the database specialist
    """
    result = db_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content


# Wrap Documents Agent as a tool
@tool(
    "documentation_specialist",
    description="Search TechHub documentation for product specs, policies, warranties, and setup instructions",
)
def call_documentation_specialist(query: str) -> str:
    """Call the documentation specialist subagent.

    Args:
        query: The question to ask the documentation specialist
    """
    result = docs_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content

**💡 Best Practice - Using `@tool` Decorator for Subagents**

Following [LangChain's multi-agent guidelines](https://docs.langchain.com/oss/python/langchain/multi-agent#where-to-customize), we use the `@tool(name, description)` pattern:

```python
@tool(
    "database_specialist",
    description="Query TechHub database for order status, product prices, and customer order history"
)
def database_specialist(query: str) -> str:
    ...
```

**Why this approach?**
1. **Name** - Clear and descriptive (not generic like `query_db`). Indicates this is a specialist agent.
2. **Description** - Concise but complete. Lists key capabilities to guide supervisor routing.
3. **Docstring** - Now minimal, just documents the parameter. No duplicate info!

**Key insight:** The supervisor only sees the tool name and description - not the sub-agent's internal prompts or tools!

In [9]:
# Create Supervisor Agent
supervisor_agent = create_agent(
    model=llm,
    tools=[call_database_specialist, call_documentation_specialist],
    system_prompt="""You are a supervisor for TechHub customer support.

Your role is to interact with the customer to understand their questions and route them to the appropriate specialists with additional context as needed:
- Use database_specialist for order status, product prices, and customer order history
- Use documentation_specialist for product specs, policies, and general information

You can use multiple tools if needed to fully answer the question.
Always provide helpful, complete responses to customers.""",
    checkpointer=MemorySaver(),
)

## 5. Test Simple Routing

Let's test the supervisor with queries that need just ONE specialist:

In [10]:
print("Query 1: Order status (should route to Database Agent)")
print("=" * 60)

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What's the status of order ORD-2025-0030?"}
        ]
    },
    config=config,
)

result["messages"][-1].pretty_print()
print("\n💡 Check LangSmith traces to see: Supervisor → database_specialist → DB Agent")

Query 1: Order status (should route to Database Agent)
================================== Ai Message ==================================

Your order **ORD-2025-0030** is currently being **processed** and has not yet been shipped. 

Here are the details:
- **Item:** AirPods Pro (2nd Generation) - Quantity: 1
- **Price:** $240.11
- **Status:** Processing
- **Tracking Number:** Not available yet

Once your order is ready to ship, you'll receive a tracking number via email so you can monitor its delivery. Is there anything else you'd like to know about this order?

💡 Check LangSmith traces to see: Supervisor → database_specialist → DB Agent


In [11]:
print("\nQuery 2: Product question (should route to Documents Agent)")
print("=" * 60)

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What's included in the box with the Logitech MX Keys keyboard?",
            }
        ]
    },
    config=config,
)

result["messages"][-1].pretty_print()
print(
    "\n💡 Check LangSmith traces to see: Supervisor → documentation_specialist → Documents Agent"
)


Query 2: Product question (should route to Documents Agent)
================================== Ai Message ==================================

The **Logitech MX Keys Wireless Keyboard** comes with:

1. **The keyboard** itself
2. **Logitech Bolt USB receiver** - for wireless connectivity
3. **USB-A to USB-C charging cable** (1.5m) - to charge the keyboard
4. **User documentation** - setup guides and reference materials

Additionally, while not included in the box, you can download the free **Logitech Options+ software** from logitech.com to access advanced features.

The keyboard will need an initial charge (about 3 hours for a full charge using the included cable), and you can start using it right away even while it's charging.

Is there anything else you'd like to know about this keyboard?

💡 Check LangSmith traces to see: Supervisor → documentation_specialist → Documents Agent


## 6. Test Multi-Agent Coordination

Now the more interesting part - queries that require BOTH sub-agents!

**Example:** Query that requires both Database AND Documents subagents, can run with PARALLEL execution

In [12]:
print("Query 3: Requires both Database AND Documents agents - parallel")
print("=" * 60)

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Is the MacBook Air in stock? What type of processor does it have? And if I buy it, what's the return policy?",
            }
        ]
    },
    config=config,
)

result["messages"][-1].pretty_print()
print("\n💡 Check LangSmith traces to see:")
print(
    "   Supervisor → database_specialist → DB Agent → documentation_specialist → Documents Agent"
)

Query 3: Requires both Database AND Documents agents - parallel
================================== Ai Message ==================================

Perfect! Here's the information you need:

## Stock Status ✓
Yes, the **MacBook Air M2 (13-inch, 256GB) is in stock** and ready to purchase at **$1,199.00**.

## Processor
The MacBook Air features the **Apple M2 chip** with:
- **8-core CPU** for powerful performance
- **8-core GPU** for graphics processing
- **8GB unified memory** (shared between CPU and GPU)

This chip is excellent for everyday tasks, students, and professionals. Additional features include up to 18 hours of battery life, a fanless design for silent operation, and a lightweight 2.7-pound design.

## Return Policy
Here's what you need to know:

- **Opened Electronics**: You have **14 days** from delivery to return it if it's in good working condition with all accessories included
- **Unopened Electronics**: You have **30 days** from delivery for a full refund (original packag

**Example:** Query that requires SEQUENTIAL agent execution - output from first agent feeds into second agent

This demonstrates true agent orchestration where the supervisor can't parallelize!

In [13]:
print("Query 4: Requires SEQUENTIAL coordination (DB → Documents)")
print("=" * 60)

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = supervisor_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I bought a monitor in my last order (ORD-2024-0063). Is the MacBook Air compatible with it?",
            }
        ]
    },
    config=config,
)

result["messages"][-1].pretty_print()
print("\n💡 Check LangSmith traces to see SEQUENTIAL flow:")

Query 4: Requires SEQUENTIAL coordination (DB → Documents)
================================== Ai Message ==================================

Perfect! **Yes, your Dell UltraSharp 27" 4K Monitor is fully compatible with MacBook Air.** Here's what you need to know:

### Compatibility Confirmed ✓
Your monitor comes with **multiple connection options**, giving you flexibility:

**Best Connection Options for MacBook Air:**
1. **USB-C cable (included)** - The easiest option! Just plug it directly into your MacBook Air's USB-C port. This provides video output and can charge your Mac simultaneously.
2. **HDMI cable (included)** - Alternative option if you prefer. You may need a USB-C to HDMI adapter (we carry these for $15-25 if needed).
3. **DisplayPort cable** - Also included for maximum flexibility.

### What You Get
Your Dell UltraSharp monitor includes all the necessary cables, so you can connect it right away without purchasing additional adapters (unless you prefer HDMI and don't have an

### Try More Examples

Test the supervisor with various queries to see routing in action:

In [ ]:
# Try these queries and watch the traces in LangSmith!

test_queries = [
    # ===== SIMPLE ROUTING (Single Agent) =====
    # Database Agent only
    "What's the status of order ORD-2024-0123?",
    "How much does the MacBook Air cost and is it in stock?",
    "What items did I order in ORD-2024-0063?",
    # Documents Agent only (product specs)
    "What are the specs of the Sony WH-1000XM5 headphones?",
    "Tell me about the ports on the Dell UltraSharp monitor",
    # Documents Agent only (policies)
    "What's your return policy?",
    "How long is the warranty on electronics?",
    # ===== PARALLEL COORDINATION (Both Agents Independently) =====
    "Is the MacBook Air in stock? What processor does it have?",
    "How much is the Logitech MX Keys and what are its key features?",
    # ===== SEQUENTIAL COORDINATION (Output → Input) =====
    # DB → DB: Get order items, then check availability
    "What did I buy in order ORD-2024-0063 and is it still available?",
    # DB → Documents: Get order items, then get specs
    "I ordered something in ORD-2023-0023. Can you tell me about its features?",
    # DB → Documents: Get order items, then get compatibility/policy info
    "I bought a monitor in order ORD-2024-0063. Is the MacBook Air compatible with it?",
]

# Uncomment to test:
# for i, query in enumerate(test_queries, 1):
#     print(f"\n[{i}/{len(test_queries)}] Query: {query}")
#     print("=" * 70)
#     result = supervisor_agent.invoke(
#         {"messages": [{"role": "user", "content": query}]},
#         config={"configurable": {"thread_id": str(uuid.uuid4())}}
#     )
#     print(result["messages"][-1].content)
#     print()

print("✓ Try uncommenting the code above to test all queries!")
print(f"\n📊 Total test queries: {len(test_queries)}")
print("\n💡 Query Types:")
print("   • Simple routing: 7 queries (single agent handles entire request)")
print("   • Parallel coordination: 2 queries (both agents work independently)")
print("   • Sequential coordination: 3 queries (one agent's output feeds another)")
print("\n🔍 Watch LangSmith traces to see:")
print("   - Which specialist the supervisor chooses")
print("   - Which tools each specialist uses")
print("   - Sequential dependencies in action!")

---

## 📦 Code Refactoring Note

The agents we built in this section (Database Agent, Documents Agent, and Supervisor) have been **refactored into the `agents/` directory** as reusable factory functions:

- `agents/db_agent.py` - Database Agent factory
- `agents/docs_agent.py` - Documents Agent factory
- `agents/supervisor_agent.py` - Supervisor Agent factory

**Why factory functions?**
- Fresh checkpointer for each instantiation (no state pollution)
- Clean imports and reusability across notebooks
- Standard LangChain pattern

In **Section 4**, we'll import these agents rather than redefining them:
```python
from agents import create_db_agent, create_docs_agent, create_supervisor_agent
```

This demonstrates how to architect production-ready, reusable code! 🏗️

## Key Takeaways

#### What We Built

1. **Specialized Sub-Agents**
   - Database Agent: Expert at structured data queries
   - Documents Agent: Expert at document search
   - Each agent has focused tools and expertise

2. **Supervisor Pattern**
   - Sub-agents wrapped as tools (`@tool` decorator)
   - Supervisor routes queries to appropriate specialist(s)
   - Can orchestrate **parallel** or **sequential** coordination

3. **Coordination Patterns**
   - **Simple routing**: Single agent handles entire query
   - **Parallel coordination**: Multiple agents work independently on different parts
   - **Sequential coordination**: Output from one agent feeds into another (true orchestration!)

4. **Benefits of Multi-Agent Architecture**
   - **Separation of concerns** - Each agent has clear responsibility
   - **Easier debugging** - Traces show which agent handled what
   - **Maintainability** - Update one agent without affecting others
   - **Scalability** - Easy to add new specialist agents


#### Viewing Traces in LangSmith

Go to your LangSmith project to see:
- **Routing decisions** - Which agent(s) the supervisor called
- **Tool executions** - What tools each agent used
- **Message flow** - Complete conversation tree
- **Timing** - How long each step took
- **Sequential dependencies** - See when agents must run in order

This visibility makes multi-agent systems much easier to debug!

#### What's Next: Section 4 (Advanced Track)

In **Section 4**, we'll use **LangGraph primitives** to build even more sophisticated workflows:
- Custom state management beyond messages
- Conditional routing based on state
- `interrupt()` for Human-in-the-Loop
- Customer verification workflows

**Key decision point:**
- Use `create_agent` for standard agent workflows ✅
- Use LangGraph primitives for complex orchestration needs ✅

#### Looking Ahead to Module 2

Notice how we're still limited by having only specific, pre-defined tools:
- Can't answer: "What products has customer CUST-001 bought?" (need SQL JOINs)
- Can't answer: "Show me all orders over $500" (need flexible queries)
- Can't answer: "Which customers bought MacBooks?" (need complex SQL)

In **Module 2**, we'll upgrade the Database Agent to a **SQL Generation Agent** that can:
- Generate dynamic SQL queries for ANY question
- Handle complex JOINs and aggregations
- Dramatically expand system capabilities

This sets up a clear pedagogical win: Module 1 teaches multi-agent patterns, Module 2 shows how to make agents more powerful! 🚀